In [1]:
!pip install statsmodels

     |████████████████████████████████| 10.8MB 185kB/s eta 0:00:01
     |████████████████████████████████| 235kB 83.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [4]:
data = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [5]:
data

,PRODUCT_ID,ORDER_DATE,ORDER_QTY
0,Product_001,2019-01-02,850
1,Product_004,2019-01-02,700
2,Product_004,2019-01-02,900
3,Product_004,2019-01-02,550
4,Product_004,2019-01-02,300
...,...,...,...
225558,Product_098,2024-08-31,200
225559,Product_100,2024-08-31,800
225560,Product_100,2024-08-31,550
225561,Product_100,2024-08-31,850


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PRODUCT_ID  225563 non-null  object
 1   ORDER_DATE  225563 non-null  object
 2   ORDER_QTY   225563 non-null  int16 
dtypes: int16(1), object(2)
memory usage: 3.9+ MB


In [8]:
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PRODUCT_ID  225563 non-null  object        
 1   ORDER_DATE  225563 non-null  datetime64[ns]
 2   ORDER_QTY   225563 non-null  int16         
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 3.9+ MB


In [10]:
data

,PRODUCT_ID,ORDER_DATE,ORDER_QTY
0,Product_001,2019-01-02,850
1,Product_004,2019-01-02,700
2,Product_004,2019-01-02,900
3,Product_004,2019-01-02,550
4,Product_004,2019-01-02,300
...,...,...,...
225558,Product_098,2024-08-31,200
225559,Product_100,2024-08-31,800
225560,Product_100,2024-08-31,550
225561,Product_100,2024-08-31,850


In [11]:
data = data.set_index('ORDER_DATE')